In [22]:
print("hii")

hii


In [23]:
print("sohaib")

sohaib


In [24]:
print("hello world")

hello world


In [40]:
%pwd

'C:\\Users\\Welcome\\Desktop\\medi-chatbot-'

In [27]:
import os

# Use os.chdir() to change the directory
os.chdir(r"C:\Users\Welcome\Desktop\medi-chatbot-")

# Verify that the directory has been changed
print("Current Directory:", os.getcwd())


Current Directory: C:\Users\Welcome\Desktop\medi-chatbot-


In [6]:
%pwd

'C:\\Users\\Welcome\\Desktop\\medi-chatbot-'

In [7]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [8]:
#Extract Data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [9]:
extracted_data = load_pdf_file(data = 'Data/')

In [42]:
# extracted_data

In [11]:
#chunking operation
# split the data into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 20) 
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 5860


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#download the embeddings from the hugging face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings
embeddings = download_hugging_face_embeddings()

C:\Users\Welcome\AppData\Local\Temp\ipykernel_15592\3392259316.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Welcome\Desktop\medi-chatbot-\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [15]:
#cheking here embedding hua download huahai ya nhi and checking their dimension
query_result = embeddings.embed_query("hello world")
print("Length", len(query_result))


Length 384


In [16]:
#checking vector
# query_result

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.environ.get('HUGGINGFACEHUB_API_TOKEN')


In [30]:
#initialize pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medi-chatbot2"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [31]:
# Set your Hugging Face API key and pinecone 
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [32]:
# Embed each chunk and upsert the embeddings into your pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [43]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [44]:
docsearch

In [46]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [47]:
retrieved_docs = retriever.invoke("what is fever")

In [49]:
retrieved_docs
#this dont give the complete answer that's why we need to llm 

[Document(id='94242930-101a-478e-8cfd-95b4cdcc9e1a', metadata={'page': 69.0, 'source': 'Data\\Medical_book.pdf'}, page_content='fever in children. This disease is most often caused by\ntypes 3 and 7. Symptoms, which appear suddenly and\nusually disappear in less than a week, include:\n• inflammation of the lining of the eyelid (conjunctivitis)\n•f e v e r\n• sore throat (pharyngitis)\n• runny nose\n• inflammation of lymph glands in the neck (cervical\nadenitis)\nGALE ENCYCLOPEDIA OF MEDICINE 256\nAdenovirus infections\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 56'),
 Document(id='5fa1169f-1d39-42db-82ae-5dae54831d06', metadata={'page': 60.0, 'source': 'Data\\Medical_book.pdf'}, page_content='(38°–40°C). In addition, a general ill feeling, muscle\naches,headache, chills, and loss of appetite may be felt.\nDiagnosis\nIf lymphangitis is suspected, the person should call\nhis or her doctor immediately or go to an emergency\nroom. Acute lymphangitis could be diagnosed by the\nfamily do

In [50]:
from langchain import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace
from langchain.llms import HuggingFaceHub
from langchain.prompts import ChatPromptTemplate




# Initialize the Hugging Face LLM
# Initialize LLM

# Hugging Face Hub ke liye free model configuration
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={
        "temperature": 0.7,
        "max_new_tokens": 250,
        "top_p": 0.9
    }
)


# System prompt
system_prompt = (
    "You are a highly knowledgeable assistant tasked with providing detailed and accurate answers. "
    "For the given query, please answer in detail, covering all aspects of the topic. "
    "If the query asks for multiple components (e.g., causes, symptoms, treatments), provide a thorough breakdown for each. "
    "Be as detailed as possible within the token limit, and ensure that you give examples where applicable. "
    "Always reference the retrieved context when answering, and if more information is needed, request clarification politely. "
    "If the information in the context is insufficient, clearly state that and ask for further details."
    "\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)



C:\Users\Welcome\AppData\Local\Temp\ipykernel_15592\3725394950.py:19: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [51]:
# Create a chain for generating responses
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)


In [53]:
response = rag_chain.invoke({"input": "early stages of cancer?"})
print(response["answer"])


Stage 1—The cancer is no larger than 2 cm (0.8 in) and no cancer cells are found in the lymph nodes. Stage 2—The cancer is between 2 cm and 5 cm, and the cancer has spread to the lymph nodes. Stage 3A—Tumor is larger than 5 cm (2 in) or is small- er than 5 cm, but has spread to the lymph nodes, which have grown into each other. Stage 3B—Cancer has spread to tissues near the breast, (local invasion), or to lymph nodes inside the chest it often becomes systemic or widespread early in the course of the disease. By the time one can feel a lump in the breast it is often 0.4 inches, or one centimeter, in size and contains roughly a million cells.
